In [8]:
import sqlalchemy as sa
from sqlalchemy import orm

In [9]:
engine = sa.create_engine('sqlite:///heroes.db', echo=True)
connection = engine.connect()

In [10]:
class Base(orm.DeclarativeBase):
    pass


In [11]:
class BattleEvents(Base):
    __tablename__ = "battle_events"
    battle_event_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)
    battle_event_type_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    rubies_gained: orm.Mapped[int] = orm.mapped_column(sa.Integer)
    timestamp: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    battle_participant_id: orm.Mapped[int] = orm.mapped_column(
        sa.ForeignKey("battle_participants.battle_participant_id"))

    def __repr__(self):
        return f'{type(self).__name__}({self.rubies_gained}, {self.timestamp}, {self.battle_participant_id})'


class BattleParticipants(Base):
    __tablename__ = 'battle_participants'

    battle_participant_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)
    user_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    battle_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, nullable=False)
    hero_id: orm.Mapped[int] = orm.mapped_column(sa.ForeignKey('heroes.hero_id'), nullable=False)

    def __repr__(self):
        return f'{type(self).__name__}({self.battle_participant_id}, {self.user_id}, {self.battle_id})'


class Heroes(Base):
    __tablename__ = 'heroes'

    name: orm.Mapped[str] = orm.mapped_column(sa.String, nullable=False, unique=True)
    hero_id: orm.Mapped[int] = orm.mapped_column(sa.Integer, primary_key=True)

    def __repr__(self):
        return f'{type(self).__name__}({self.name})'

In [16]:

# Wyświetl nazwę bohatera oraz w ilu walkach wziął udział
# SELECT heroes.name, Count(be.battle_event_id) AS TotalEvents
# FROM heroes AS h
# JOIN battle_participants AS bp ON bp.hero_id = h.hero_id
# JOIN battle_events AS be ON be.battle_partcipants_id = bp.battle_paricipant_id
q = (sa.select(Heroes.name, sa.func.count(BattleEvents.battle_event_id).label("TotalEvents"))
     .join(BattleParticipants, BattleParticipants.hero_id == Heroes.hero_id)
     .join(BattleEvents, BattleEvents.battle_participant_id == BattleParticipants.battle_participant_id).group_by(Heroes.name).order_by(sa.desc("TotalEvents")))
dc = connection.execute(q)
d = dc.fetchall()
print(d)

2025-05-13 21:57:55,840 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-13 21:57:55,842 INFO sqlalchemy.engine.Engine SELECT heroes.name, count(battle_events.battle_event_id) AS "TotalEvents" 
FROM heroes JOIN battle_participants ON battle_participants.hero_id = heroes.hero_id JOIN battle_events ON battle_events.battle_participant_id = battle_participants.battle_participant_id GROUP BY heroes.name ORDER BY "TotalEvents" DESC
2025-05-13 21:57:55,845 INFO sqlalchemy.engine.Engine [generated in 0.00551s] ()
[('Crareek', 3687), ('Pamble', 2054), ('Huzzt', 2036), ('Tan', 1946), ('Grimm', 1820), ('Jade', 1780), ('Hillstomp', 1736), ('Mako', 1719), ('Grrdy', 1528)]


In [17]:
# Podaj bochatera który zabił najwięcej innych bohaterów

